In [ ]:
##Longhand - M. Cook - 2022
##Takes Bag-of-Words and returns multi-user, immersive visualization
##https://github.com/Cook4986/Longhand

In [ ]:
#NER and Parts-of-Speech extraction from HTR bag-of-words
#See: https://github.com/Cook4986/SAEF_OCR for more info on HTR pre-processing
#Entity types: https://github.com/mchesterkadwell/named-entity-recognition/blob/main/2-named-entity-recognition-of-henslow-data.ipynb
import spacy
from collections import Counter

#declarations
nlp = spacy.load("en_core_web_sm")
BoW = "/Users/matthewcook/Dropbox/DS/Zaritt/Woman in Chains.txt"
nounOut = open("/Users/matthewcook/Dropbox/Viz/Longhand/nouns.txt", "w")
    
###parse BoW with Spacy
with open(BoW, encoding="utf-8") as file:
    iliad = file.read()
document = nlp(iliad)
document.text

#collate nouns in BoW input
nouns = []
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.text)

#Entity Frequency
word_freq = Counter(nouns)
common_words = word_freq.most_common(100)
print(common_words)
for noun in common_words:
    noun.strip("'()[]")
    print(noun)
    nounOut.write(str(noun))
    nounOut.write("\n")

#close files
file.close()
nounOut.close()
print("\n")
print("have a nice day")

In [ ]:
#query Sketchfab with nouns and/or named entities and generate list of uids for use in Blender
import json
import requests
from requests.exceptions import RequestException
import re
import time

#declarations
SKETCHFAB_API_URL = "https://api.sketchfab.com/v3/search?type=models"
API_TOKEN = '046f786a59eb4112b9b89ba26a85f85f'
common = open("/Users/matthewcook/Dropbox/Viz/Longhand/nouns.txt", "r")
uid_Out = open("/Users/matthewcook/Dropbox/Viz/Longhand/uid.txt", "w")
words = []
uids = []
count = 0
start = time.time()

#extract noun strings from common words doc
for line in common.readlines():
    word = line.split()
    words.append(word[0]) 

#Sketchfab API payload function 
##From https://sketchfab.com/developers/data-api/v3/python#example-python-model
def _get_request_payload(*, data=None, files=None, json_payload=False):
    """Helper method that returns the authentication token and proper content type depending on
    whether or not we use JSON payload."""
    data = data or {}
    files = files or {}
    headers = {'Authorization': 'Token {}'.format(API_TOKEN)}
    if json_payload:
        headers.update({'Content-Type': 'application/json'})
        data = json.dumps(data)
    return {'data': data, 'files': files, 'headers': headers}

#query sketchfabs with nouns in BoW and return/write list of uids
for word in words:
    word = word.strip("(),'")
    print("Searching: " + str(word))
    print("\n")
    query = ("&q="+(str(word))+"&downloadable=true")
    search_endpoint = f'{SKETCHFAB_API_URL + query}'
    payload = _get_request_payload() 
    response = requests.get(search_endpoint, **payload)
    data = response.json()
    for item in range(len(data['results'])):
        uid=(data['results'][item]['uid'])
        name = (str((data['results'][item]['name'])))
        print("the following model been located: ")
        print(str((data['results'][item]['name']))+(" \nuid: ")+(data['results'][item]['uid']))
        print("\n")
        uids.append(name +", "+ uid)
for uid in uids:
    uid_Out.write(uid + "\n")
    
#close program
common.close()
uid_Out.close()
end = time.time()
print(str(end - start) + " seconds elapsed" )
print("\n")
print("have a nice day")



In [ ]:
##Launches Blender from terminal and initiates python script, above
!/Applications/Blender.app/Contents/MacOS/Blender --python /Users/matthewcook/Longhand/operator_file_import.py


In [ ]:
##TO DO:
#initiate Sketchfab queries in Blender using Blender Python API
#place search results in the scene, without overlap, using frequencies
#export scene (gltf) for use in hubs
#automate scene change function in Hubs, based on newly created gltfs